<a href="https://colab.research.google.com/github/NPCA-TEAM/COVID-19/blob/main/Scripts/%209%20-%20P%C3%93S_PROCESSAMENTO_covid_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalação das bibliotecas (necessário reiniciar ambiente)

In [ ]:
!pip install darts 

In [ ]:
!pip install matplotlib==3.1.3

##Importação das bibliotecas

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import glob
import os
import sys
import time
import torch
import json

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

from datetime import datetime
from darts import timeseries
from darts.dataprocessing.transformers import Scaler

from darts.models import (    
    NBEATSModel,
    TCNModel,
    TransformerModel,
    TFTModel 
)

from darts.metrics import mape, rmse, r2_score, coefficient_of_variation
from darts.utils.timeseries_generation import datetime_attribute_timeseries

import warnings
warnings.filterwarnings('ignore')
import logging
logging.disable(logging.CRITICAL)
torch.manual_seed(1); np.random.seed(1)  # for reproducibility

#Permitir acesso ao Gdrive
from google.colab import drive
drive.mount('/content/drive')

##Leitura do dataset

In [ ]:
#Seleciona o arquivo da base de dados recentemente atualizado na pasta
files_list = glob.glob(path_dataset + '*') 

#Seleciona o ultimo arquivo que foi criado e adicionado na pasta
file_path = max(files_list, key=os.path.getctime) 
file_path

In [ ]:
#Leitura e exibição do Arquivo
data_set = pd.read_excel(file_path, sheet_name='Base')

data_set

In [ ]:
#Retira colunas desnecessárias para o treinamento
data = data_set.drop(data_set.columns[4:], axis=1, inplace=False)

#print(data)
#Converte coluna de data (que é string) para datetime 
data['Ano'] = pd.to_datetime(data['Ano'],format='%Y')
#print(data)

##Pós-processamento e armazenamento de dados gerados

In [ ]:
def pos_processing_and_saveFig(sclr_Serie,ser_scaled,ser_pred,ser_hist_fcast,serie_name,name_model): 
  #Volta para a escala original
  ser_back = sclr_Serie.inverse_transform(ser_scaled)
  ser_back_pred = sclr_Serie.inverse_transform(ser_pred)
  ser_back_hist_fcast = sclr_Serie.inverse_transform(ser_hist_fcast)

  #Visualização das predições
  fig, ax = plt.subplots(figsize=(8,4));
  fig.suptitle(target_variable, fontsize=12);
  ser_back.plot()
  ser_back_pred.plot(label = name_model + ' forecast ahead')
  ser_back_hist_fcast.plot(label = name_model + ' historical forecast')

  #Salva o plot com figura 'output.png'
  plt.savefig(path_figs + serie_name + '_' + name_model + '.png')

  return ser_back, ser_back_pred, ser_back_hist_fcast

In [ ]:
def pos_processing_and_saveForecasts_CSV(serie_back, serie_back_pred, serie_back_hist_fcast, serie_name,name_model):

  #Tratamento para exportação em CSV
  df_export_Serie = serie_back.pd_dataframe()
  df_export_Hist_fcast = serie_back_hist_fcast.pd_dataframe()
  df_export_Pred = serie_back_pred.pd_dataframe()

  df_export_Pred.index.names = ['Ano']
  df_export_Hist_fcast = df_export_Hist_fcast.rename({'0': serie_name + '_real'}, axis= 'columns') 
  df_export_Hist_fcast.index.names = ['Ano']

  df_prediction = pd.concat([df_export_Hist_fcast[:-num_years_of_slide_val], df_export_Pred])

  df_export_Serie[serie_name+'_predicted'] = df_prediction[serie_name+'_real']

  df_export_Serie = df_export_Serie.reset_index()

  ultimo_ano = df_export_Pred.reset_index()['Ano'].values[-num_years_of_slide_val]

  ultimo_valor = df_export_Pred.reset_index()[serie_name+'_real'].values[-num_years_of_slide_val]

  new_row = {'Ano':ultimo_ano, serie_name+'_predicted':ultimo_valor}

  df_export_Serie = df_export_Serie.append(new_row, ignore_index=True)
  print(df_export_Serie)

  #Exportação do dado
  df_export_Serie.to_csv(path_csv+serie_name+'_'+name_model+'.csv', index=False)

  return df_export_Serie

In [ ]:
def generate_nameSerie(variable_, region_): # gerar nomes das séries com os parâmetros: variável e região
  #Gerar nome da serie.
  caracters_to_remove = "()1234567890"

  variable = variable_
  region = region_
  
  for char in caracters_to_remove:
    region = region.replace(char, "")

  region = region.replace(" I", "_I")
  region = region.replace(" ", "")

  variable = variable.replace('Coeficiente de detecção','')
  variable = variable.replace(' ','')      
  serie_name = region + '_' + variable

  return serie_name

In [ ]:
variables = [''] # simulando variáveis



#==========Para testes============
models = ['N-BEATS'] 

# BLOCK_LSTM - BLOCK_GRU


#variables = ['Coeficiente de detecção geral'] 
#regions = ['MARAJÓ I ( 9 )'] 
#models = ['N-BEATS']

regions = ['MARAJÓ II (7)',
          'METROPOLITANA I (05)',
          'METROPOLITANA II (09)',
          'METROPOLITANA III (22)',
          'RIO CAETÉS (16)',
          'TAPAJÓS (06)',
          'TOCANTINS (09)',
          'XINGU (09)']

#===================================

for a in variables:
  for b in regions:
    for c in models: 

      serie_name = generate_nameSerie(a, b)

      print('Variavel: {}. Regiao: {}. Modelo: {}. Nome da serie: {}'.format(a, b, c, serie_name))
      print('------------------------------------------')

      target_variable = a
      target_region = b
      name_model = c

      #Pós-processamento e salvar as figuras. 
      serie_back, serie_back_pred, serie_back_hist_fcast = pos_processing_and_saveFig(scaler_Serie,serie_scaled,serie_pred,serie_hist_fcast,serie_name,name_model)
      
      #Pós-processamento e salvar as CSVs.
      export_Serie = pos_processing_and_saveForecasts_CSV(serie_back, serie_back_pred, serie_back_hist_fcast, serie_name, name_model)